## Worm-Like Chain (WLC) Simulation Tutorial for UAMMD-structured

## Introduction
This notebook demonstrates how to create and run a simulation of polymer chains using the Worm-Like Chain (WLC) model in UAMMD-structured. The WLC model is commonly used to describe semi-flexible polymers. We'll walk through the process step-by-step, explaining each part of the simulation setup in detail.

## Setup

First, let's import the necessary libraries:

In [16]:
import os
import numpy as np
import pyUAMMD

# os: Used for directory operations
# numpy: Used for numerical operations and array manipulations
# pyUAMMD: The Python interface for UAMMD-structured

## Simulation Parameters

Here we define the key parameters for our WLC simulation:

In [24]:
# Number of beads
nBeads = 30

# Bead diameter (and bond length)
sigma = 1.0

# Calculate box size (add some extra space)
L = nBeads*sigma + 2.0*sigma

# Simulation parameters
timeStep = 0.001
frictionConstant = 1.0

# Total number of simulation steps
nSteps = 20000

# Frequency of information output
nStepsInfo = 1000

# Frequency of trajectory output
nStepsOutput = 10000

# Force constants for bonds and angles
Kb = 100.0  # Bond strength
Ka = 100.0  # Angle strength (bending rigidity)

print("Creating a WLC simulation with:")
print(f" - Number of beads: {nBeads}")
print(f" - Bead diameter: {sigma}")
print(f" - Box size: {L}")
print(f" - Time step: {timeStep}")
print(f" - Friction constant: {frictionConstant}")
print(f" - Total number of steps: {nSteps}")
print(f" - Bond strength (Kb): {Kb}")
print(f" - Bending rigidity (Ka): {Ka}")
print(f" - Output information every {nStepsInfo} steps")
print(f" - Output trajectory every {nStepsOutput} steps")

Creating a WLC simulation with:
 - Number of beads: 30
 - Bead diameter: 1.0
 - Box size: 32.0
 - Time step: 0.001
 - Friction constant: 1.0
 - Total number of steps: 20000
 - Bond strength (Kb): 100.0
 - Bending rigidity (Ka): 100.0
 - Output information every 1000 steps
 - Output trajectory every 10000 steps



## Creating the Simulation

Now, let's create our simulation object and set up its various components:

In [18]:
# Initialize the simulation object
simulation = pyUAMMD.simulation()

# Set up the system information
simulation["system"] = {
    "info": {
        "type": ["Simulation", "Information"],
        "parameters": {"name": "WormLikeChain"}
    }
}

# Define global parameters
simulation["global"] = {
    # Set the unit system (in this case, we're using reduced units)
    "units": {"type": ["Units", "None"]},

    # Define particle types
    "types": {
        "type": ["Types", "Basic"],
        "labels": ["name", "mass", "radius", "charge"],
        "data": [["A", 1.0, sigma/2.0, 0.0]]
    },

    # Set the ensemble (NVT: constant Number of particles, Volume, and Temperature)
    "ensemble": {
        "type": ["Ensemble", "NVT"],
        "labels": ["box", "temperature"],
        "data": [[[L, L, L], 1.0]]
    }
}

# Set up the integrator (Langevin dynamics)
simulation["integrator"] = {
    "bbk": {
        "type": ["Langevin", "BBK"],
        "parameters": {
            "timeStep": timeStep,
            "frictionConstant": frictionConstant
        }
    },
    # Define the integration schedule
    "schedule": {
        "type": ["Schedule", "Integrator"],
        "labels": ["order", "integrator", "steps"],
        "data": [[1, "bbk", nSteps]]
    }
}

## Initialize Particle Positions and Topology

We'll place the polymer chains in the simulation box and set up the topology:

In [ ]:
simulation["state"] = {
    "labels": ["id", "position"],
    "data": []
}

simulation["topology"] = {
    "structure": {
        "labels": ["id", "type", "modelId"],
        "data": []
    }
}

particleId = 0
for j in range(int(nBeads)):
    # Place beads along the z-axis, centered at the origin
    simulation["state"]["data"].append([particleId, [0.0, 0.0, sigma*j - nBeads*sigma/2.0]])
    simulation["topology"]["structure"]["data"].append([particleId, "A", i])
    particleId += 1

## Define Force Field

Set up the bonded interactions for our WLC model:

In [ ]:
simulation["topology"]["forceField"] = {}

# Set up harmonic bonds
simulation["topology"]["forceField"]["bonds"] = {
    "type": ["Bond2", "Harmonic"],
    "parameters": {},
    "labels": ["id_i", "id_j", "K", "r0"],
    "data": []
}

particleId = 0
for j in range(int(nBeads) - 1):
    simulation["topology"]["forceField"]["bonds"]["data"].append([particleId, particleId + 1, Kb, sigma])
    particleId += 1

# Set up angle interactions (Kratky-Porod potential for bending rigidity)
simulation["topology"]["forceField"]["angles"] = {
    "type": ["Bond3", "KratkyPorod"],
    "parameters": {},
    "labels": ["id_i", "id_j", "id_k", "K"],
    "data": []
}

particleId = 0
for j in range(int(nBeads) - 2):
    simulation["topology"]["forceField"]["angles"]["data"].append([particleId, particleId + 1, particleId + 2, Ka])
    particleId += 1


## Configure Simulation Steps

Define what operations to perform during the simulation:

In [21]:
simulation["simulationStep"] = {
    # Output simulation information periodically
    "info": {
        "type": ["UtilsStep", "InfoStep"],
        "parameters": {"intervalStep": nStepsInfo}
    },
    # Save trajectory data periodically
    "output": {
        "type": ["WriteStep", "WriteStep"],
        "parameters": {
            "intervalStep": nStepsOutput,
            "outputFilePath": "output",
            "outputFormat": "sp"
        }
    }
}

## Write the Simulation File

Finally, let's write our simulation to a JSON file:

In [22]:
print()
print("Writing simulation file...")
simulation.write("simulation.json")
print("Simulation file created successfully!")
print()

04/09/2025 18:02:49 - pyUAMMD - WARNING - Writing with legacy 



Writing simulation file...
Simulation file created successfully!



## Running the Simulation

To run the simulation, you would typically use the UAMMD-structured executable with the generated JSON file.

In [23]:
print()
print("You can run the code now using: UAMMDlauncher simulation.json")
simulation.run()


You can run the code now using: UAMMDlauncher simulation.json


[WARNING] UAMMD-structured Python wrapper is not compatible with UAMMD-structured self restarting mechanism
[MESSAGE] ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
[MESSAGE] ╻ ╻┏━┓┏┳┓┏┳┓╺┳┓
[MESSAGE] ┃ ┃┣━┫┃┃┃┃┃┃ ┃┃ Version: 2.5
[MESSAGE] ┗━┛╹ ╹╹ ╹╹ ╹╺┻┛
[MESSAGE] Compiled at: May 30 2025 09:48:24
[MESSAGE] Compiled in double precision mode
[MESSAGE] Computation started at Thu Sep  4 18:02:50 2025

[MESSAGE] [System] CUDA initialized
[MESSAGE] [System] Using device: NVIDIA GeForce RTX 4070 Ti with id: 0
[MESSAGE] [System] Compute capability of the device: 8.9
[MESSAGE] ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ ━ 
[MESSAGE] [ExtendedSystem] (system) Name: WormLikeChain
[MESSAGE] [ExtendedSystem] (system) Seed: 1757001770139834578
[MESSAGE] [GlobalDataBase] Fundamental not specified, using default fundamental, "Time"
[WARNING] [Time] No timeStep specified, using 0.0 as default.
[MESSAGE] [Basic] Loaded type A, mass: 1.000000, radius: 0.500000, charge: 0.0000

## Conclusion

This tutorial demonstrated how to set up and prepare a simulation of Worm-Like Chains using UAMMD-structured. We covered:
1. Defining simulation parameters, including polymer-specific parameters
3. Creating the simulation object
4. Setting up the system, global parameters, and integrator
5. Initializing particle positions and topology for multiple polymer chains
6. Configuring the simulation topology and force field, including bonded and angle interactions
7. Setting up simulation steps for output
8. Writing the simulation file
9. Running the simulation

Next steps could include:
- Analyzing the output trajectory
- Visualizing the polymer chains
- Calculating polymer properties such as end-to-end distance or radius of gyration
- Modifying the simulation parameters to explore different polymer lengths, stiffnesses, or environmental conditions